## **협업 필터링 (CF)**

어떤 아이템에 대해서 비슷한 취향을 가진 사람들은 다른 아이템에 대해서도 비슷한 취향을 가지고 있을 것이라고 가정해 볼 수 있다. 이러한 아이디어로 만들어진 추천 알고리즘이 협업 필터링(Collaborative Filtering)이라고 한다.

### **유사도 지표**
* **상관계수** : 연속된 값에 적용할 수 있는 가장 기본적인 유사도 지표로는 상관계수(correlation coefficient)가 있다. 상관계수는 이해하기 쉬운 유사도 측정치이기는 하지만 협업 필터링에서 사용하는 경우, 늘 좋은 결과를 가져오지는 못하는 것으로 알려져 있다.

* **코사인 유사도** : 코사인 유사도에서는 각 아이템을 하나의 차원으로 보고 사용자의 평가값을 좌표값으로 본다. 그렇게 되면 각 사용자의 평가값을 벡터로 해서, 두 사용자 간의 벡터의 각도(코사인 값)를 구할 수 있다.
```python
from sklearn.metrics.pairwise import cosine_similarity
# ...
user_similiarity = cosine_similarity(rating_matrix, rating_matrix)
```

* 만일 데이터가 이진값(binary)을 갖는다면 타니모토 계수(Tanimoto coefficient) 혹은, 자카드 계수(Jaccard coefficient) 등을 주로 사용한다.

### **이웃을 고려한 CF**
* CF 알고리즘을 개선할 수 있는 한 가지 방법은 사용자 중에서 유사도가 높은 사용자를 선정해서 이웃의 크기를 줄이는 것이다. 이렇게 대상 사용자의 유사도가 높은 사람의 평가만 사용하면 당연히 예측의 정확도가 올라갈 것으로 예상해 볼 수 있다.
* 크게 두 가지 방법이 있는데, 하나는 이웃의 크기를 미리 정해높고 추천 대상 사용자와 가장 유사한 K명을 선택하는 KNN 방식과, 또 다른 하나는 특정 유사도의 기준을 정해놓고, 이 기준을 충족하는 모든 이웃을 선택하는 threshold 방식이 있다. 주로 전자를 사용한다.
* 주로 CF 알고리즘의 정확도를 최대로 하는 이웃의 크기가 존재한다. 이웃의 크기를 너무 크게 하면 집단별 취향의 차이가 없어지고, best-seller 방식과 크게 다를 바가 없게 된다. 반대로, 이웃의 크기가 지나지체 작으면 유사도가 매우 높은 소수의 이웃의 평가만을 사용하게 된다. 이렇게 되면 소수의 평가에 지나치게 의존하게 되어 예측치의 신뢰성이 낮아지게 된다. 

### **사용자의 평가 경향을 고려한 CF**
* CF의 정확도를 더 개선시키는 방법 중의 하나는 사용자의 평가 경향 (user_bias)을 고려해서 예측치를 조정하는 것이다. 사용자에 따라서 평가를 전체적으로 높게 하는 사람이 있는 반면에, 평가를 전체적으로 낮게 하는 사람이 있는 등, 사람에 따라 평가경향이 다르다. 이러한 평가 경향을 고려하기 위해 CF 알고리즘에 대입되는 데이터는 실제 평점이 아닌, 평점에서 평균을 뺀 평점 편차이다.
```python
rating_mean = rating_matrix.mean(axis = 1) # 각 사용자마다 평점 평균을 구한다.
rating_bias = (rating_matrix.T - rating_mean).T # 평점에서 평균을 뺀 편차를 구한다.
```

### **신뢰도 가중**
* CF의 정확도를 개선시키는 마지막 방법은 신뢰도 가중(significance weights)이 있다. A에 대해서 영화 평점을 예측할 때, A와 공통으로 평가한 영화의 수가 특정 수 이상인 사용자들만 CF 알고리즘에 활용하는 것이다. A와 공통으로 평가한 영화의 수가 많을 수록, 둘 사이의 유사도 지수의 신뢰도가 높다고 할 수 있으므로, 이러한 점을 반영한 것이다.
```python
no_rating = movie_ratings.isnull() # movie_id를 평점 매기지 않은 유저들
common_counts = counts[user_id] 
low_significance = common_counts < SIG_LEVEL # 신뢰도가 일정 수준 미만인 유저들
none_rating_idx = movie_ratings[no_rating | low_significance].index # 인덱스 추출
movie_ratings = movie_ratings.drop(none_rating_idx)
sim_scores = sim_scores.drop(none_rating_idx)
```

### **IBCF와 UBCF**
* 이 코드에서는 사용자를 기준으로 비슷한 취양의 이웃을 선정하는 방식을 사용하였다. 이런 방식을 UBCF(User-Based CF)라고 한다. 반대로 아이템을 기준으로 하는 아이템 기반 CF(Item-Based CF)도 가능하다.
* UBCF는 각 사용자 별로 맞춤형 추천을 하기 때문에 데이터가 풍부한 경우 정확한 추천이 가능하다. 반대로 IBCF는 정확도는 떨어지지만 사용자별로 따로 따로 계산을 하지 않기 때문에 계산이 빠르다는 장점이 있다. 또한 UBCF는 정확할 때에는 매우 정확하지만, 터무니 없는 경우도 상당히 있는 데 비해, IBCF는 그럴 위험이 적다.






In [20]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/u.data', names = r_cols, sep='\t', encoding = 'latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)

i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'umknown', 'Action',
          'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 
          'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'war', 'western']
movies = pd.read_csv('/content/u.item', sep='|', names = i_cols, encoding = 'latin-1')

x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, stratify = y)

# 사용자와 영화이름, 평점을 행렬 구조로 바꾼다.
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

# RMSE 함수
def RMSE(y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred)) ** 2))

# CF 알고리즘의 손실값을 계산하는 함수
def score(model, neighbor_size=0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

# 사용자 간의 코사인 유사도를 계산한다.
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = matrix_dummy.index, columns = matrix_dummy.index)

# 사용자의 평가 경향을 고려하기 위해 평균을 뺀다.
rating_mean = rating_matrix.mean(axis = 1)
rating_bias = (rating_matrix.T - rating_mean).T

# 신뢰도 가중을 위해, 사용자 간에 공통으로 평가한 영화의 수를 계산한다.
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary1, rating_binary2)
counts = pd.DataFrame(counts, index = rating_matrix.index, columns = rating_matrix.index).fillna(0)

def CF_knn_bias_sig(user_id, movie_id, neighbor_size):
  if movie_id in rating_bias:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = rating_matrix[movie_id].copy()

    no_rating = movie_ratings.isnull() # movie_id를 평점 매기지 않은 유저들
    common_counts = counts[user_id] 
    low_significance = common_counts < SIG_LEVEL # 신뢰도가 일정 수준 미만인 유저들
    none_rating_idx = movie_ratings[no_rating | low_significance].index # 인덱스 추출

    movie_ratings = movie_ratings.drop(none_rating_idx)
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      prediction = np.dot(movie_ratings, sim_scores) / sim_scores.sum()
      prediction = prediction + rating_mean[user_id]
    else:
      if len(sim_scores) > MIN_RATINGS:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        
        user_idx = np.argsort(sim_scores) # 유사도 순으로 정렬
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
        prediction = prediction + rating_mean[user_id]
      else:
        prediction = rating_mean[user_id]
  else:
    prediction = rating_mean[user_id]
  
  return prediction

SIG_LEVEL = 3
MIN_RATINGS = 2
print("손실 값 : %d" % score(CF_knn_bias_sig, 30))

# 특정 유저에 대해서 n_items 개수의 영화를 추천해준다.
def recom_movie(user_id, n_items, neighbor_size=30):
  user_movie = rating_matrix.loc[user_id].copy()
  for movie in rating_matrix:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0 # 이미 평점이 맺어진 영화는 값을 0으로 바꿔서 추천 후보에서 제외한다.
    else:
      user_movie.loc[movie] = CF_knn_bias_sig(user_id, movie, 30)

  movie_sort = user_movie.sort_values(ascending=False)[:n_items] # 영화 평점순으로 정렬
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

print(recom_movie(user_id=2, n_items=5, neighbor_size=30))


손실 값 : 3
movie_id
119                      Striptease (1996)
1449                Golden Earrings (1947)
64      What's Eating Gilbert Grape (1993)
963            Month by the Lake, A (1995)
187               Full Metal Jacket (1987)
Name: title, dtype: object
